# Red Social con CUDA C++

Este notebook implementa una red social simple utilizando CUDA para operaciones paralelas.

**Características:**
- Personas, empresas y publicaciones
- Sistema de seguimiento y bloqueo
- Likes/Dislikes en publicaciones
- Hashtags
- Red de influencia
- Visibilidad de publicaciones

**Instrucciones:**
1. Asegúrate de tener GPU habilitada: `Runtime > Change runtime type > GPU`
2. Ejecuta las celdas en orden

## 1. Verificar GPU disponible

In [ ]:
!nvidia-smi

## 2. Crear archivo social_network.cu

**Opción A:** Sube manualmente el archivo `social_network.cu` desde tu computadora usando el panel de archivos de Colab (carpeta a la izquierda).

**Opción B:** Ejecuta la siguiente celda para crear el archivo directamente (recomendado).

In [ ]:
%%writefile social_network.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <string.h>
#include <algorithm>

#define MAX_USERS 1000
#define MAX_POSTS 2000
#define MAX_TEXT_LEN 256
#define MAX_HASHTAG_LEN 32
#define THREADS_PER_BLOCK 256

// ============================================================================
// ESTRUCTURAS DE DATOS
// ============================================================================

// Tipos de usuario
enum UserType { PERSON = 0, COMPANY = 1 };

// Tipo de interacción con publicación
enum Interaction { NONE = 0, LIKE = 1, DISLIKE = 2 };

// Estructura para personas (SoA - Structure of Arrays)
struct Persons {
    int count;
    int ids[MAX_USERS];
    char names[MAX_USERS][64];
};

// Estructura para empresas
struct Companies {
    int count;
    int ids[MAX_USERS];
    char names[MAX_USERS][64];
};

// Estructura para publicaciones
struct Posts {
    int count;
    int ids[MAX_POSTS];
    char texts[MAX_POSTS][MAX_TEXT_LEN];
    char hashtags[MAX_POSTS][MAX_HASHTAG_LEN];
    int author_ids[MAX_POSTS];
    UserType author_types[MAX_POSTS];
    int original_post_id[MAX_POSTS];  // -1 si es original, sino ID del post original
};

// Relaciones entre usuarios (matrices de adyacencia)
// 0 = no relación, 1 = relación
struct Relations {
    // Persona -> Persona
    int person_follows_person[MAX_USERS][MAX_USERS];
    int person_blocks_person[MAX_USERS][MAX_USERS];

    // Persona -> Empresa
    int person_follows_company[MAX_USERS][MAX_USERS];
    int person_is_client[MAX_USERS][MAX_USERS];
    int person_works_at[MAX_USERS][MAX_USERS];
    int person_blocked_by_company[MAX_USERS][MAX_USERS];

    // Empresa -> Empresa
    int company_follows_company[MAX_USERS][MAX_USERS];
    int company_recommends_company[MAX_USERS][MAX_USERS];
    int company_blocks_company[MAX_USERS][MAX_USERS];

    // Empresa -> Persona
    int company_blocks_person[MAX_USERS][MAX_USERS];
};

// Interacciones con publicaciones
struct PostInteractions {
    Interaction person_interactions[MAX_USERS][MAX_POSTS];
    Interaction company_interactions[MAX_USERS][MAX_POSTS];
};

// ============================================================================
// KERNELS CUDA - OPERACIONES BÁSICAS
// ============================================================================

// Kernel para contar seguidores de una persona
__global__ void count_person_followers_kernel(int person_idx, int* relations, int num_persons, int* result) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    __shared__ int local_count[THREADS_PER_BLOCK];
    local_count[threadIdx.x] = 0;

    if (tid < num_persons) {
        if (relations[tid * MAX_USERS + person_idx] == 1) {
            local_count[threadIdx.x] = 1;
        }
    }

    __syncthreads();

    // Reducción paralela
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            local_count[threadIdx.x] += local_count[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(result, local_count[0]);
    }
}

// Kernel para contar seguidores de una empresa
__global__ void count_company_followers_kernel(int company_idx,
                                               int* person_follows_company,
                                               int* company_follows_company,
                                               int num_persons,
                                               int num_companies,
                                               int* result) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    __shared__ int local_count[THREADS_PER_BLOCK];
    local_count[threadIdx.x] = 0;

    // Contar personas que siguen a la empresa
    if (tid < num_persons) {
        if (person_follows_company[tid * MAX_USERS + company_idx] == 1) {
            local_count[threadIdx.x] = 1;
        }
    }
    // Contar empresas que siguen a la empresa
    else if (tid < num_persons + num_companies) {
        int company_tid = tid - num_persons;
        if (company_follows_company[company_tid * MAX_USERS + company_idx] == 1) {
            local_count[threadIdx.x] = 1;
        }
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            local_count[threadIdx.x] += local_count[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(result, local_count[0]);
    }
}

// Kernel para contar likes de una publicación
__global__ void count_post_likes_kernel(int post_idx,
                                        Interaction* person_interactions,
                                        Interaction* company_interactions,
                                        int num_persons,
                                        int num_companies,
                                        int* likes,
                                        int* dislikes) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    __shared__ int local_likes[THREADS_PER_BLOCK];
    __shared__ int local_dislikes[THREADS_PER_BLOCK];
    local_likes[threadIdx.x] = 0;
    local_dislikes[threadIdx.x] = 0;

    if (tid < num_persons) {
        Interaction inter = person_interactions[tid * MAX_POSTS + post_idx];
        if (inter == LIKE) local_likes[threadIdx.x] = 1;
        else if (inter == DISLIKE) local_dislikes[threadIdx.x] = 1;
    }
    else if (tid < num_persons + num_companies) {
        int company_tid = tid - num_persons;
        Interaction inter = company_interactions[company_tid * MAX_POSTS + post_idx];
        if (inter == LIKE) local_likes[threadIdx.x] = 1;
        else if (inter == DISLIKE) local_dislikes[threadIdx.x] = 1;
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            local_likes[threadIdx.x] += local_likes[threadIdx.x + stride];
            local_dislikes[threadIdx.x] += local_dislikes[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(likes, local_likes[0]);
        atomicAdd(dislikes, local_dislikes[0]);
    }
}

// Kernel para encontrar publicaciones con un hashtag
__global__ void find_posts_by_hashtag_kernel(char (*hashtags)[MAX_HASHTAG_LEN],
                                             int num_posts,
                                             const char* target_hashtag,
                                             int* result_indices,
                                             int* result_count) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < num_posts) {
        bool match = true;
        for (int i = 0; i < MAX_HASHTAG_LEN; i++) {
            if (hashtags[tid][i] != target_hashtag[i]) {
                match = false;
                break;
            }
            if (hashtags[tid][i] == '\0') break;
        }

        if (match && hashtags[tid][0] != '\0') {
            int idx = atomicAdd(result_count, 1);
            if (idx < MAX_POSTS) {
                result_indices[idx] = tid;
            }
        }
    }
}

// Kernel para verificar si una persona puede ver una publicación
__global__ void check_visibility_kernel(int post_idx,
                                       int author_id,
                                       int* person_follows_person,
                                       int* person_blocks_person,
                                       int num_persons,
                                       int* can_view) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < num_persons) {
        // El autor siempre puede ver su propia publicación
        if (tid == author_id) {
            can_view[tid] = 1;
            return;
        }

        bool is_follower = (person_follows_person[tid * MAX_USERS + author_id] == 1);
        bool is_blocked = (person_blocks_person[author_id * MAX_USERS + tid] == 1);

        // Puede ver si es seguidor y no está bloqueado
        if (is_follower && !is_blocked) {
            can_view[tid] = 1;
            return;
        }

        // Buscar si es seguidor de un seguidor (grado 2)
        for (int i = 0; i < num_persons; i++) {
            bool i_follows_author = (person_follows_person[i * MAX_USERS + author_id] == 1);
            bool tid_follows_i = (person_follows_person[tid * MAX_USERS + i] == 1);
            bool i_blocked_by_author = (person_blocks_person[author_id * MAX_USERS + i] == 1);

            if (i_follows_author && !i_blocked_by_author && tid_follows_i) {
                can_view[tid] = 1;
                return;
            }
        }

        can_view[tid] = 0;
    }
}

// ============================================================================
// FUNCIONES HOST (Inicialización, queries, etc.)
// ============================================================================
// NOTA: Por brevedad, el resto del código se mantiene igual que en social_network.cu
// Aquí solo se muestra la estructura principal. Copiar el resto del archivo original.

**IMPORTANTE:** La celda anterior solo muestra la estructura de kernels. Necesitas copiar el archivo completo `social_network.cu` desde tu computadora local al entorno de Colab.

Usa el panel de archivos (icono de carpeta a la izquierda) y arrastra el archivo `social_network.cu` allí.

## 3. Compilar el código CUDA

In [ ]:
# Compilar con nvcc
!nvcc -o social_network social_network.cu -std=c++11

## 4. Ejecutar el programa

In [ ]:
!./social_network

## 5. Análisis de resultados

El programa ejecutará todas las queries implementadas:

1. **Seguidores**: Cantidad de seguidores por persona y empresa
2. **Reacciones**: Likes y dislikes por publicación
3. **Top Posts**: Top 5 con más y menos likes
4. **Bloqueados**: Seguidores bloqueados
5. **Recomendaciones**: Empresas que recomiendan a otras
6. **Hashtags**: Análisis de hashtags y publicaciones
7. **Mejores clientes**: Clientes que más interactúan
8. **Empresas destacadas**: Por likes y recomendaciones
9. **Visibilidad**: Quién puede ver cada publicación
10. **Red de influencia**: Seguidores hasta grado N

## Notas de Implementación

### Estructuras de datos
- **SoA (Structure of Arrays)**: Mejor rendimiento en GPU que AoS
- **Matrices de adyacencia**: Para relaciones entre usuarios
- **Kernels paralelos**: Reducción, conteo, búsqueda

### Operaciones CUDA
- **Reducción paralela**: Para contar seguidores y reacciones
- **Atomic operations**: Para sincronización de resultados
- **Shared memory**: Para optimizar accesos

### Limitaciones
- MAX_USERS = 1000
- MAX_POSTS = 2000
- Datos hardcodeados para demostración